# Object oriented programming

Rust's design is highly influenced by functional languages, and features we discuss here (traits and polymorphism) are more close to their equivalents in Haskell and ML languages than to C++ and Java. But since Rust supports mutable variables, it can provide an object oriented like experience.

In Rust we have C structs:

In [2]:
struct Human {
    name: &'static str,
    age: u32,
}

let hamid = Human { name: "hamid", age: 22 };
hamid.age

22

We can define functions in the namespace of types, with `impl` keyword:

In [3]:
impl Human {
    // Rust doesn't have constructor, but we can use functions for it:
    fn new(name: &'static str, age: u32) -> Human {
        Human { name, age } // shorthand for `name: name, age: age`
    }

    fn new_baby(name: &'static str) -> Human {
        Human { name, age: 0 }
    }
}

let danial = Human::new("danial", 24);
danial.age

24

Namespaced functions can be completely unrelated to the type:

In [4]:
impl Human {
    fn a_completely_unrelated_function(x: i32) -> i32 {
        x + 5
    }
}

Human::a_completely_unrelated_function(10)

15

First argument of such function can be have a special keyword, `self`, which make the function a method:

In [5]:
impl Human {
    fn say_hello(self: &Human) {
        println!("{} says hello!", self.name);
    }
}

(&hamid).say_hello();
// compiler will add & automatically
hamid.say_hello();
// normal call is possible
Human::say_hello(&danial);

hamid says hello!
hamid says hello!
danial says hello!


Unlike normal functions, method type is restricted. It can be `Human`, `&Human`, `&mut Human`, `Box<Human>` and similar:

In [6]:
impl Human {
    fn unrelated(self: &i32) {
        println!("{}", self);
    }
}

Error: invalid `self` parameter type: &i32

`&mut self` is a shorthand for `self: &mut Human`:

In [7]:
impl Human {
    fn pass_a_year(&mut self) {
        self.age += 1;
    }
}

let mut baby = Human::new_baby("ali");
baby.pass_a_year();
baby.pass_a_year();
baby.pass_a_year();
baby.age

3

## Privacy and encapsulation